# <center> Identify Customer Segments- Machine Learning </center>

## Table of Contents

1. <a href="#Intro">Introduction</a>
2. <a href="#Trans">Feature Transformation</a>
    * <a href="#Scale"> 2.1 - Apply Feature Scaling </a>
    * <a href="#Dimen"> 2.2 - Perform Dimensionality Reduction </a>
    * <a href="#Inter"> 2.3 - Interpret Principal Components </a>
3. <a href="#Clust">Clustering </a>
    * <a href="#Gener"> 3.1 - Apply Clustering to General Population </a>
    * <a href="#Custo"> 3.2 - Apply All Steps to the Customer Data </a>
    * <a href="#Compa"> 3.3 - Compare Customer Data to Demographics Data </a>

<a id = 'Intro'></a>
## Introduction

Based on the [data cleansing](http://localhost:8888/notebooks/Identify_Customer_Segments_Exploration.ipynb)  notebook, we now have a solid understanding of the data under consideration. _In this notebook_, we load the previously cleaned data then we proceed with the <u>_feature transformation_</u> as well as the <u>_clustering parts_</u> of this project. We conclude this analysis by applying the whole pipeline for an another dataset containing the same demographic data about specific customers in a trial of analysing the common patterns associated with the general population. Hopefully, this process will lead to a better understanding of the customer behaviours in shadow of the available data about general population. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from helpers import do_pca, scree_plot, pca_results, map_weights,fit_kmeans
from sklearn.cluster import KMeans

<a id = 'Clean'></a>
### 2 -  Load cleaned dataset

The complete data cleaning procedure and detailed justification for each step is given on the [Data Cleansing notebook](http://localhost:8888/notebooks/Identify_Customer_Segments_Exploration.ipynb). To simplify this notebook, we just load the resulted dataframe. 

In [ ]:
azdias_cleaned = pd.read_csv('Udacity_AZDIAS_Subset_Cleaned.csv', index_col = 0)
azdias_cleaned.head()

In [ ]:
original_feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv', delimiter = ';')
original_feat_info.index = original_feat_info.attribute
original_feat_info.drop('attribute', axis = 1, inplace = True)

<a id = 'Trans'></a>
## 3 - Feature Transformation

<a id = 'Scale'></a>
### 3.1 -  Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. In this substep, we perform a standard scaling so that all the features have a mean of 0 and a standard deviation of 1. 

In [ ]:
# Apply feature scaling to the general population demographics data.
Sclaer_obj = StandardScaler()
azdias2 = pd.DataFrame(Sclaer_obj.fit_transform(azdias_cleaned), columns = azdias_cleaned.columns)

<a id = 'Dimen'></a>
### 3.2 - Perform Dimensionality Reduction

On the scaled data, we are now ready to apply dimensionality reduction techniques. Below is a summary of upcoming procedure. 

- Using sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class, we apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, we should not set any parameters (so all components are computed).
- Checking out the ratio of variance explained by each principal component as well as the cumulative variance explained. In addition, we plot the cumulative values. Based on the findings, we select a value for the number of transformed features to retain for the clustering part.
- Finally, we re-fit a PCA instance to perform the decided-on transformation on the selected number of the features.

In [ ]:
# Apply PCA to the whole scaled data.
pca, X_pca = do_pca(n_components = azdias_cleaned.shape[1], data = azdias2)

In [ ]:
# Investigate the variance accounted for by each principal component.
vals,cumvals = scree_plot(pca)

Having completed the PCA analysis using all the features, we retain specific number of components to facilitate the upcoming clustering as possible. Thus, we need to keep number of the features which results in explaining specific threshold of the original data's variance.  

In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.
threshold_PCA_retain = .8
comp_to_retain = np.where(cumvals <= threshold_PCA_retain)[0][-1]
pca, X_pca = do_pca(n_components = comp_to_retain, data = azdias_cleaned)
FinalPCAresults = pca_results(azdias_cleaned, pca)

In [ ]:
FinalPCAresults.shape

Selecting to retain the principal components explaining `80%` of the variance in the dataset lead to have new data with only `83` columns. 

<a id = 'Inter'></a>
### 3.3 -  Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

In a nutshell, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, we should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list.
- We should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, we wrote a function `map_weights` to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.
weights2 = map_weights(FinalPCAresults, 2)
weights2

In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.
weights3 = map_weights(FinalPCAresults, 3)
weights3

### Discussion 2.3: Interpret Principal Components

(Double-click this cell and replace this text with your own text, reporting your observations from detailed investigation of the first few principal components generated. Can we interpret positive and negative values from them in a meaningful way?)

<a id = 'Clust'></a>
## 4 -  Clustering

<a id = 'Gener'></a>
### 4.1 - Apply Clustering to General Population

Having assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, we will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

A summary of the upcoming procedure is given below
-  Performing k-means clustering on the PCA-transformed data using sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans).
- Computing the average difference from each point to its assigned cluster's center. 
- Performing the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Using this fact, we select a final number of clusters in which to group the data.
- Having selected a final number of clusters to use, we re-fit a KMeans instance to perform the clustering operation.

In [ ]:
X_pca[0:1000].shape

In [ ]:
centers_1 = np.arange(20,30)
scores_1 = []
for k in centers_1:
    model = KMeans(n_clusters = k)
    model.fit(X_pca[0:1000])
    scores_1.append(np.abs(model.score(X_pca[0:1000])))

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
plt.plot(centers_1, scores_1, linestyle='--', marker='o', color='b');
plt.xlabel('K');
plt.ylabel('SSE');
plt.title('SSE vs. K');

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.
k_best = 25
labels,Kmeans_obj = fit_kmeans(X_pca[0:1000], k_best) #fit kmeans to get the labels

In [ ]:
# Plot the original data with clusters
plt.scatter(azdias2['ALTERSKATEGORIE_GROB'].iloc[0:1000], azdias2['FINANZ_ANLEGER'].iloc[0:1000], c=labels, cmap='Set1');

### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

<a id = 'Custo'></a>
### 5.2 -  Apply All Steps to the Customer Data

Now we have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. In the last step of the project, we will interpret how the general population fits apply to the customer data.

A summary of the upcoming procedure is given below:
- Loading the customer data
- Applying the same cleansing pipeline done with the general population data
- Applying the same feature transformation steps (i.e., scaling and pca) using the objects obtained previosuly. 
- Finally, clustering the tranformed data using the same clustering object. 

In [ ]:
def impute_nan_most_frequent_category(DataFrame,ColName):
    """
    Impute categorical data with mode values
    """
    most_frequent_category=DataFrame[ColName].mode()[0]
    # replace nan values with most occured category
    DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
    DataFrame[ColName + "_Imputed"].fillna(most_frequent_category,inplace=True)
    
def clean_data(df, feat_info):
    """
    Perform feature trimming, re-encoding, and engineering for demographics data
    
    INPUT: 
        - df: Demographics DataFrame
        - feat_info: Information about df columns
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """

    print('Data cleansing pipeline...')
    print('df shape is {}'.format(df.shape))
    ### Main cleaning steps:    
    ## 1- convert missing value codes into NaNs, ...
    print('\t1-Replacing missing values per column with Nan...')
    a = list(feat_info['missing_or_unknown'])
    ToNan = [i.strip('][,').split(',') for i in a]
    for i,j in zip(df.columns, ToNan):
        NanIndices = df.query('{} == {}'.format(i, j)).index
        df.at[NanIndices,i] = np.NaN

    print('\t\tReplacing done')
    # ------------------------------------------------------------------------------------------------ 
    ## 2- Remove selected columns and rows 
    print('\t2-Removing the outliars columns and rows...')
    # 2.1- Perform an assessment of how much missing data there is in each column of the dataset.
    
    MissingValues = pd.DataFrame([df.isna().sum(),(df.isna().sum() / df.shape[0]) * 100], 
                                    index = ['Values','Per'], 
                                 columns = df.columns).T
    Threshold = MissingValues.describe()['Per']['75%']
    Todelete = MissingValues.query('Per > {}'.format(Threshold))
    DeletedColumns = df.loc[:,Todelete.index]
    print('Deleted columns shape is {}'.format(DeletedColumns.shape))
    # 2.2- Remove the outlier columns from the dataset
    
    df.drop(Todelete.index, axis = 1 , inplace = True)
    feat_info.drop(Todelete.index, axis = 0 , inplace = True)
    # 2.3- Perform an assessment of how much missing data there is in each row of the dataset

    MissingValuesRows = pd.DataFrame([df.isna().sum(axis = 1),(df.isna().sum(axis = 1) / df.shape[1]) * 100],
                                     columns = df.index, index = ['Value','Per']).T
    
    # 2.4- Divide the data into two subsets based on the number of missing values in each row.
    ThresholdRows = MissingValuesRows.describe()['Per']['75%']
    TodeleteRows = MissingValuesRows.query('Per > {}'.format(ThresholdRows))
    DeletedRows = df.loc[TodeleteRows.index,:]
    print('Deleted rows shape is {}'.format(DeletedRows.shape))
    df.drop(TodeleteRows.index, axis = 0 , inplace = True)
    df.reset_index(inplace = True , drop = True)
    print('\t\tRemoving the outliars columns and rows done')
    # ------------------------------------------------------------------------------------------------
    ## 3- Select, Re-encode, and Engineer column values.
    print('\t3-Feature Engineering...')
    # 3.1- Assess categorical variables: which are binary, which are multi-level, and which one needs to be re-encoded?
    
    cat_var = df.loc[:,list(feat_info.query('type =="categorical"').index)]
    unique_per_cat = [(i,cat_var[i].nunique()) for i in cat_var.columns]
    bin_cat = [i for i in cat_var.columns if cat_var[i].nunique() == 2 and pd.api.types.is_string_dtype(cat_var[i])]
    print('bin_cat is {}'.format(bin_cat))
    Multi_cat = [i for i in cat_var.columns if cat_var[i].nunique() > 2]
    To_OH = pd.concat([df.loc[:, Multi_cat], df.loc[:, bin_cat]], axis = 1)
    print('Missing values by column : {}'.format(To_OH.isna().sum()))
    # 3.2- Impute missing values for data frame to be onehot encoded
    for k,v in To_OH.isna().sum().items():
        if v > 0:
            impute_nan_most_frequent_category(To_OH,k)
            To_OH.drop(k, axis = 1, inplace = True)
    print('3.2: df shape is {}'.format(df.shape))    
    # 3.3- Re-encode categorical variable(s) to be kept in the analysis.
    print('To_OH columns are {}'.format(To_OH.columns))
    print('To_OH shape {}'.format(To_OH.shape))
    print('{}'.format(To_OH.nunique()))
    OH_bin_obj = preprocessing.OneHotEncoder().fit(To_OH)
    OH_cat = OH_bin_obj.transform(To_OH)
    OH_cat = pd.DataFrame(OH_cat.toarray(), columns = OH_bin_obj.get_feature_names())
    print('3.3: shape is {}'.format(df.shape))
    # 3.4- Drop categorical features
    df.drop(cat_var , axis = 1 , inplace = True)
    feat_info.drop(cat_var, axis = 0, inplace = True)
    print('3.4: df shape is {}'.format(df.shape))
    # 3.5- Append one hot encoded data 
    df = pd.concat([df, OH_cat], axis = 1)
    print('3.5: df shape is {}'.format(df.shape))
    print('3.5: OH_cat shape is {}'.format(OH_cat.shape))
    # 3.6- Return the cleaned dataframe.
    # Investigate "PRAEGENDE_JUGENDJAHRE" and engineer new variables.
    df['movement'] = df.PRAEGENDE_JUGENDJAHRE % 2 
    Wealth = [int(i[0]) if type(i) == str else i for i in df.CAMEO_INTL_2015]
    LifeStage = [int(i[1])  if type(i) == str else i for i in df.CAMEO_INTL_2015]
    CAMEO_INTL_2015 = pd.DataFrame(list(zip(Wealth, LifeStage)), columns = ['Waelth', 'LifeStage'])
    print('3.6: df shape is {}'.format(df.shape))
    # 3.7- Drop mixed features
    df.drop(feat_info.query('type =="mixed"').index, axis = 1, inplace = True)
    # 3.8- Append new features
    df = pd.concat([df,CAMEO_INTL_2015] , axis = 1)
    print('3.7: df shape is {}'.format(df.shape))
    print('\t\tFeature Engineering done')
    print('df shape is {}'.format(df.shape))
    # ------------------------------------------------------------------------------------------------
    ## 4- Remove all the missing values
    print('\t4-Removing missing values...')
    df.dropna(inplace = True)
    df.reset_index(inplace = True ,drop = True)
    print('\t\tRemoving missing values done.')
    print('df shape is {}'.format(df.shape))
    # ------------------------------------------------------------------------------------------------
    print('Cleaning pipeline done')
    return df

In [2]:
# Load in the customer demographics data.
customers = pd.read_csv('Udacity_CUSTOMERS_Subset.csv', delimiter = ';')

In [4]:
customers.GEBAEUDETYP.unique()

array([ 1., nan,  8.,  2.,  3.,  4.,  6.])

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.
customers_cleaned = clean_data(customers,original_feat_info)

In [ ]:
azdias_cleaned.shape, customers_cleaned.shape

In [ ]:
set(customers_cleaned.columns) - set(azdias_cleaned.columns) 

In [ ]:
customers_scaled = pd.DataFrame(Sclaer_obj.transform(customers_cleaned), columns = customers_cleaned.columns)
customers_scaled

In [ ]:
customers_transformed = pca.transform(customers_scaled)
customers_labels = Kmeans_obj.transform(customers_transformed)

<a id = 'Compa'></a>
### 5.3 - Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.



In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.